In [ ]:
import cv2 
import numpy as np
import os
def get_image_files(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]
def rotated_true_labels(folder_path, num_rotations) -> str:
    return "Not yet implimented"


In [1]:
def preprocess_images(img_folder_path, label_folder_path):
    #get every image in the path
    image_files = get_image_files(img_folder_path)
    for image_file in image_files:
        #get the image_file name without extension
        base_name = os.path.splitext(os.path.basename(image_file))[0]
        image = cv2.imread(image_file)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
        #now time for some data augmenting
            #let's add rotated version of the image to the dir as 0 thru 3 
        rotated_image = blurred_image
        augmented_images = [blurred_image]
        for i in range(3):
            rotated_image = cv2.rotate(rotated_image, cv2.ROTATE_90_CLOCKWISE)
            augmented_images.append(rotated_image)

        #finally, write the image to preprocessed/images/base_name_0 thru 3 .jpg
        for i, augmented_image in enumerate(augmented_images):
            cv2.imwrite(f'preprocessed/images/{base_name}_{i}.jpg', augmented_image)
            with open(f'preprocessed/labels/{base_name}_{i}.txt', 'w') as f:
                f.write(rotated_true_labels(label_folder_path, i))

        #now, write the rotated image lables to preprocessed/labels/base_name_0 thru 3 .txt
        

        

    

